In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,990 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:13 http://security.ubuntu.com/ub

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-01 03:46:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.41MB/s    in 0.2s    

2022-10-01 03:46:36 (5.41 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...|2015-08-31 00:00:00|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...| 

In [ ]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'string'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'timestamp')]

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
reviewdf = df.select(["review_id", "customer_id","product_id","product_parent",to_date("review_date","yyyy-MM-dd").alias("review_date")]).drop_duplicates(["review_id"])
reviewdf.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000D29OXI03C|   13683294|B0037TV0IW|     846180035| 2014-01-31|
|R1000EGOSOZ2HM|   16191136|B00D5GC78O|     945426222| 2015-04-02|
|R1000FQ3QQ1HII|   11709408|B0071U4T40|     551702715| 2012-05-17|
|R1000GNGJ2RDWH|   43138655|B00TY1GLVI|     474167162| 2015-05-28|
|R1000NV62D5QYK|   51108537|B00OQ19QYA|      13246935| 2015-03-07|
|R1000TR62BELR6|    9874125|B000NVZE5M|     447732471| 2007-11-22|
|R1000UR9DW4LP8|   24698139|B00845NI8E|     677032789| 2013-03-03|
|R1000WOS9M5I8V|    1711944|B00JW492KY|     876646749| 2015-05-05|
|R1000XKIAFW7AL|   31067634|B00869HYYM|     241152579| 2014-12-16|
|R100117T0OTEJ6|   11208593|B00BSDSDSO|     656140075| 2013-06-19|
|R10011R459NQXP|   11256941|B0099YB2Q4|     957442756| 2013-05-06|
|R100182P4RYK45|   32894605|B00MXWFUQC|     322621570| 2015-06

In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
|   42801586|             1|
|    9565734|             2|
|   15829398|             1|
|   38247118|             1|
|   32478248|             2|
|   48114630|             1|
|   23085063|             1|
|   32787070|             3|
|   43515569|             1|
|    4919528|             2|
|    5088547|             2|
|   41852407|             3|
|   49703087|             1|
|   12713799|             1|
|   36728141|             8|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).drop_duplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|011040047X|Purple Hard Case ...|
|0110400550|Pink & White 3d M...|
|0133070565|Samsung Galaxy S ...|
|0214614700|Cool Summer Breez...|
|0214714705|Cool Summer Breez...|
|0471295418|New Trading Dimen...|
|0594011868|Nook Simple Touch...|
|0594017580|Jonathan Adler Pu...|
|0594033896|Nook Color Clear ...|
|0594033926|Barnes & Noble De...|
|0594450705|Nook Vara Stylus ...|
|0594459451|Barnes & Noble No...|
|059446790X|2-way Cover Stand...|
|0594467926|Barnes and Noble ...|
|0594479525|Barnes & Noble Va...|
|0594482569|Nook HD+ 9 Inch P...|
|059448426X|Composition Cover...|
|0594513189|NOOK HD 7" Cover ...|
|0594513294|NOOK HD Austen Qu...|
|0594513324|NOOK HD 7" Cover ...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [2]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://module-16-challenge.cg2ms2tdc9fn.us-east-1.rds.amazonaws.com:5432/module-16-challenge"
config = {"user":"postgres", 
          "password": "Dasani595!", 
          "driver":"org.postgresql.Driver"}

In [3]:
# Write review_id_df to table in RDS
reviewdf.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

NameError: ignored

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

Py4JJavaError: ignored